# Marshall Fire 

![](https://www.mercurynews.com/wp-content/uploads/2022/01/TDP-L-AERIAL_MARSHALL_FIRE_030.jpg?w=780)


![](https://assets.bouldercounty.org/wp-content/uploads/2021/12/cpp-marshall-fire-houses-smoke-header-20211230-1536x400.jpg)

In [1]:
import geopandas as gpd
from shapely.geometry import mapping
from pystac_client import Client
from pystac import ItemCollection

In [2]:
def make_geom(aoi_filename):
    '''creates a geom/Polygon/Coordinates - bbox - from an geojson file'''
    # read in AOI as a GeoDataFrame
    aoi = gpd.read_file(aoi_filename)

    # get the geometry of the AOI as a dictionary for use with PySTAC Client
    geom = mapping(aoi.to_dict()['geometry'][0])
    return geom

In [3]:
! ls *.geojson

marshall-fire.geojson


In [4]:
aoi_geojson_file = 'marshall-fire.geojson'

In [5]:
geom = make_geom(aoi_geojson_file)

In [19]:
def get_stac_records_sentinel_search(geom):
    # STAC API - Landsat Collection 2
    url = "https://earth-search.aws.element84.com/v0"

    # Search parameters
    params = {
        "collections": ["sentinel-s2-l2a-cogs"],
        "intersects": geom,
        "datetime": "2021-12-01/2022-01-07",
        "limit": 100,
        "query": ["eo:cloud_cover<70", "sentinel:data_coverage>10"]
    }
    cat = Client.open(url)
    search = cat.search(**params)
    
    matched = search.matched()
    print(f"{search.matched()} scenes found")
    return(search)

In [20]:
thing = get_stac_records_sentinel_search(geom)

11 scenes found


In [21]:
search_dict = thing.get_all_items_as_dict()['features']
search_items = thing.get_all_items()

In [22]:
for item in search_items:
    print(item)

<Item id=S2B_13TDE_20211229_0_L2A>
<Item id=S2A_13TDE_20211227_0_L2A>
<Item id=S2B_13TDE_20211222_0_L2A>
<Item id=S2B_13TDE_20211219_0_L2A>
<Item id=S2A_13TDE_20211217_0_L2A>
<Item id=S2A_13TDE_20211214_0_L2A>
<Item id=S2B_13TDE_20211212_0_L2A>
<Item id=S2B_13TDE_20211209_0_L2A>
<Item id=S2A_13TDE_20211207_0_L2A>
<Item id=S2A_13TDE_20211204_0_L2A>
<Item id=S2B_13TDE_20211202_0_L2A>


In [18]:
search_dict[0]['assets'].keys()

dict_keys(['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL'])

In [24]:
import yaml

from odc import stac
from pyproj import CRS
from pystac.extensions.projection import ProjectionExtension

def open_odc(items, crs=None, resolution=None):
    configuration_str = """---
        landsat-c2l2-sr:
          measurements:
            '*':
              dtype: float32
              nodata: 0
              units: 'm'
        """
    configuration = yaml.load(configuration_str, Loader=yaml.CSafeLoader)
    datasets = list(stac.stac2ds(items, configuration))
    
    crs_str = str(items[0].properties['proj:epsg'])
    crs = f'EPSG:{crs_str}'

    resolution=(-10, 10)
    data = stac.dc_load(datasets, bands=['visual'], chunks={"x": 1024, "y": 1024}, output_crs=crs, resolution=resolution)
    return data

ImportError: cannot import name 'stac' from 'odc' (unknown location)